In [1]:
import pandas as pd 
import numpy as np 
import requests
import mysql.connector
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
class WeatherApp():
    '''
    Generic Weather Class to fetch data.
    '''
    appKey = ""
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        try:
            connection = mysql.connector.connect(
            host="localhost",         # e.g., 'localhost' or IP address
            user="root",     # MySQL username
            password="root", # MySQL password
            database="weather_api"  # Database name
            )

            query = "select * from weather_api.appdetails"
            df = pd.read_sql(query, con=connection)
            connection.close()
            
            appDetails = [i for i in df['appid']]
            self.appKey = appDetails[0]

        except mysql.connector.Error as e:
            print(f"Error connecting to MySQL: {e}")


    def get_city_coordinates(self, city):
        """
        Convert a city name to latitude and longitude using the OpenWeather Geocoding API.
        
        :param city: Name of the city (e.g., "Delhi").
        :return: Tuple containing latitude and longitude.
        """
        url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={self.appKey}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            if data:
                return data[0]['lat'], data[0]['lon']  # Return latitude and longitude
            else:
                raise ValueError(f"City '{city}' not found.")
        else:
            raise Exception(f"Failed to fetch coordinates. Status code: {response.status_code}")

    
    def fetch_historical_weather(self, city, start_date, end_date):
        """
        Fetch historical weather data for a specific city and date range.
        
        :param city: Name of the city (e.g., "Delhi").
        :param start_date: Start date in "YYYY-MM-DD" format.
        :param end_date: End date in "YYYY-MM-DD" format.
        :return: DataFrame containing historical weather data for the specified date range.
        """
        # Get latitude and longitude for the city
        lat, lon = self.get_city_coordinates(city)

        # Convert start_date and end_date to datetime objects
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

        # Initialize an empty list to store weather data
        weather_data = []

        # Loop through each day in the date range
        current_date = start_date
        while current_date <= end_date:
            # Convert the current date to a Unix timestamp (seconds since 1970-01-01)
            timestamp = int(current_date.timestamp())

            # Fetch historical weather data for the current date
            url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={self.appKey}&units=metric"
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                if 'data' in data:  # Check if 'data' key exists
                    for entry in data['data']:
                        entry_date = pd.to_datetime(entry['dt'], unit='s')
                        weather_data.append({
                            "date": entry_date,
                            "temperature": entry['temp'],
                            "humidity": entry['humidity'],
                            "pressure": entry['pressure'],
                            "wind_speed": entry['wind_speed'],
                            "weather_description": entry['weather'][0]['description']
                        })
                else:
                    print(f"No data found for {current_date.date()}.")
            else:
                print(f"Failed to fetch data for {current_date.date()}. Status code: {response.status_code}")

            # Move to the next day
            current_date += timedelta(days=1)

        # Convert the list of weather data to a DataFrame
        return pd.DataFrame(weather_data)



In [3]:
class DataStorage():
    def store_weather_data_in_mysql(weather_df):
        """
        Store weather data in a MySQL database.
    
        :param weather_df: DataFrame containing weather data.
        :param db_config: Dictionary containing MySQL database connection details.
        """
        try:
            # Connect to the MySQL database
            connection = mysql.connector.connect(
                host="localhost",         # e.g., 'localhost' or IP address
                user="root",     # MySQL username
                password="root", # MySQL password
                database="weather_api"  # Database name
                )
            cursor = connection.cursor()

            # Create a table to store weather data (if it doesn't exist)
            create_table_query = """
            CREATE TABLE IF NOT EXISTS historical_weather (
                id INT AUTO_INCREMENT PRIMARY KEY,
                date DATETIME,
                temperature FLOAT,
                humidity FLOAT,
                pressure FLOAT,
                wind_speed FLOAT,
                weather_description VARCHAR(255)
            )
            """
            cursor.execute(create_table_query)

            # Insert data into the table
            insert_query = """
            INSERT INTO historical_weather (date, temperature, humidity, pressure, wind_speed, weather_description)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            for _, row in weather_df.iterrows():
                cursor.execute(insert_query, (
                    row['date'],
                    row['temperature'],
                    row['humidity'],
                    row['pressure'],
                    row['wind_speed'],
                    row['weather_description']
                ))

            # Commit the transaction
            connection.commit()
            print("Data successfully stored in MySQL database.")

        except mysql.connector.Error as error:
            print(f"Error: {error}")

        finally:
            # Close the database connection
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed.")

In [4]:
auth = WeatherApp()
weather_data = auth.fetch_historical_weather("Delhi","2024-12-01","2024-12-31")

if not weather_data.empty:
    print("Fetched weather data:")
    print(weather_data)

    # Store weather data in MySQL
    DataStorage.store_weather_data_in_mysql(weather_data)
else:
    print("No data fetched.")


Failed to fetch data for 2024-01-01. Status code: 401
Failed to fetch data for 2024-01-02. Status code: 401
Failed to fetch data for 2024-01-03. Status code: 401
Failed to fetch data for 2024-01-04. Status code: 401
Failed to fetch data for 2024-01-05. Status code: 401
Failed to fetch data for 2024-01-06. Status code: 401
Failed to fetch data for 2024-01-07. Status code: 401
Failed to fetch data for 2024-01-08. Status code: 401
Failed to fetch data for 2024-01-09. Status code: 401
Failed to fetch data for 2024-01-10. Status code: 401
Failed to fetch data for 2024-01-11. Status code: 401
Failed to fetch data for 2024-01-12. Status code: 401
Failed to fetch data for 2024-01-13. Status code: 401
Failed to fetch data for 2024-01-14. Status code: 401
Failed to fetch data for 2024-01-15. Status code: 401
Failed to fetch data for 2024-01-16. Status code: 401
Failed to fetch data for 2024-01-17. Status code: 401
Failed to fetch data for 2024-01-18. Status code: 401
Failed to fetch data for 202